# Import

In [2]:
import numpy as np

import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib import colors as mcolors
import os

import package_DBR
import PACKAGE_LAB

from importlib import reload
package_DBR = reload(package_DBR)
PACKAGE_LAB = reload(PACKAGE_LAB)

from package_DBR import *
from PACKAGE_LAB import *

NameError: name 'PID' is not defined

# Simulation Parameters

In [ ]:
TSim = 2400               
Ts = 1                      
N = int(TSim/Ts) + 1
# Working point
MV0 = 50
DV0 = 50
PV0 = 52.5             
# Parameters Step response MV
Kp = 0.22940157900053862
T1p = 235.05979493875853  
T2p =  4.990264278507082
thetap =  50.462409314789745

# Parameters Step response DV
Kd = 0.407
T1d =  150.244  
T2d =  2.7537
thetad = 6

# Process object
P = Process({})
P.parameters['kp'] = Kp
P.parameters['Tlead1'] = 0
P.parameters['Tlag1'] = T1p
P.parameters['Tlag2'] = T2p
P.parameters['theta'] = thetap
P.parameters['nInt'] = 0

# Parameters PID controller
gamma = 0.6                                             
Kc, Ti, Td = IMC_TUNING(Kp,T1p,T2p,thetap,gamma,'SOPDT')
print("Kc : ", Kc )
print("Ti : ", Ti )
print("Td : ", Td )

alpha = 1
#La puissance de chauffe est limité entre 0 et 100 %
MVMin = 0  
MVMax = 100

# 4 scénariosn de simulation

In [ ]:
scenario = "CLP+FF"

# Scenario 1 : OLP + no FF  
if scenario == "OLP+noFF":
    ManPath = {0:True, TSim: True} 
    MVManPath = {0:MV0, TSim: MV0} 
    SPPath = {0:PV0, TSim: PV0}   
    DVPath = {0: 50.0, 1100: 60, TSim: 60.0}
    ActivateFF = False
    ManFF = True

    
# Scenaro 2 : OLP + FF
elif scenario == "OLP+FF":
    ManPath = {0:True, TSim: True} 
    MVManPath = {0:MV0, TSim: MV0} 
    SPPath = {0:PV0, TSim: PV0}  
    DVPath = {0: 50.0, 1000: 60, TSim: 60.0}
    ActivateFF = True
    ManFF = True

# Scenaro 3 : CLP + no FF
elif scenario == "CLP+noFF":
    ManPath = {0:True, 500:False, TSim: False} 
    MVManPath = {0:MV0, TSim: MV0} 
    SPPath = {0: PV0+5, 1200: PV0-5, TSim: PV0-5} 
    DVPath = {0: DV0, 2000: DV0+10, TSim: DV0+10} 
    ActivateFF = False
    ManFF = False

# Scenaro 4 : CLP + FF
elif scenario == "CLP+FF":
    ManPath = {0:True, 500:False, TSim: False} 
    MVManPath = {0:MV0, TSim: MV0} 
    SPPath = {0: PV0+5, 1200: PV0-5, TSim: PV0-5} 
    DVPath = {0: DV0, 2000: DV0+10, TSim: DV0+10} 
    ActivateFF = True
    ManFF = False

# Discrete-time PID controller with feedforward structure on actual TCLab system

In [ ]:
# Initialisation 
t = []

SP = []
MV = []
Man = []
MVMan = []
MVFF = []
MVP = []
MVI = []
MVD = []
DV = []
PV = []
E = []

MV_FF_Delay = []
MV_FF_LL1 = []

lab = tclab.TCLab()

now = datetime.now()
date_time = now.strftime("%Y-%m-%d-%Hh%M")

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1)
fig.set_figheight(22)
fig.set_figwidth(22)

l1, = ax1.step([0,TSim],[-0.1,1.1],'k-',linewidth=2,label="Man",where='post')
ax1.set_ylabel('Value of Man [0 or 1]')
ax1.set_title('Closed-loop response with PID controller and feedforward on actual TCLAB system' + date_time)
ax1.legend(loc='best')

l2, = ax2.step([0,TSim],[0,100],'b-',linewidth=2,label="MV",where='post')
l9, = ax2.step([0,TSim],[0,100],'--m',linewidth=1,label="MVFF",where='post')
l3, = ax2.step([0,TSim],[0,100],'--y',linewidth=1,label="MVP",where='post')
l4, = ax2.step([0,TSim],[0,100],'--c',linewidth=1,label="MVI",where='post')
l5, = ax2.step([0,TSim],[0,100],'--r',linewidth=1,label="MVD",where='post')
ax2.set_ylabel("Value of MV [%]")
ax2.legend(loc='best')
l6, = ax3.step([0,TSim],[0,100],'k-',linewidth=2,label="SP",where='post')
l7, = ax3.step([0,TSim],[0,100],'g-',linewidth=2,label="PV",where='post')
ax3.set_ylabel("Value of PV [°C]")
ax3.legend(loc='best')
l8, = ax4.step([0,TSim],[0,100],'r-',linewidth=2,label="DV",where='post')
ax4.set_xlabel("Time [s]")
ax4.set_ylabel("Value of DV [%]")
ax4.legend(loc='best')





